In [2]:
!pip install torch_geometric
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.7 MB/s eta 0:00:0000:01


# Obiettivi
Creare una rete neurale capace di calcolare la **betweenness centrality** di un nodo all'interno di un grafo con una precisione accettabile, comparabile agli algoritmi di approssimazione tradizionali, ma con una significativa riduzione dei tempi di calcolo, quindi della complessità computazionale.

## Attività principali:
1. **Selezione del dataset (grafo) di training**
2. **Calcolo della betweenness centrality esatta** per il grafo di training
3. **Data labelling**: Aggiunta della feature *betweenness centrality* al grafo
4. **Suddivisione Dataset in test e val**
5. parallelismo -> immagine/grafo nodo/pixel
5. **Sviluppo del modello**:
    - Modello di regressione
    - Training supervisionato
6. **Training del modello**: con test e validazione
7. **Test delle prestazioni del modello**
8. **Confronto** dei risultati ottenuti con gli approcci classici


### 1. **Selezione del dataset (grafo) di training**

The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words.



In [3]:
import torch
import torch_geometric
import networkx as nx
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx

# Dataset Cora
#dataset = Planetoid(root='/tmp/Cora', name='Cora')
dataset = Planetoid(root='/tmp/Pubmed', name='Pubmed')
data = dataset[0] ##provare ad utilizzare tutti i grafi dentro dataset
#data loader di grafi.

print(type(dataset))

Processing...


<class 'torch_geometric.datasets.planetoid.Planetoid'>


Done!
/opt/conda/lib/python3.10/site-packages/torch_geometric/io/fs.py:215: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location)


In [4]:
print(data)

print(f'Numero di nodi: {data.num_nodes}')
print(f'Train mask: {data.train_mask.sum()} nodi')
print(f'Validation mask: {data.val_mask.sum()} nodi')
print(f'Test mask: {data.test_mask.sum()} nodi')

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
Numero di nodi: 19717
Train mask: 60 nodi
Validation mask: 500 nodi
Test mask: 1000 nodi


### 2. Calcolo della betweenness centrality esatta per il grafo di training


In [5]:
# Converti il grafo in formato networkx per calcolare la betweenness centrality
#G = to_networkx(data, to_undirected=True)

# Calcola la betweenness centrality dei nodi
#betweenness = nx.betweenness_centrality(G)

#data.y = torch.tensor([betweenness[i] for i in range(data.num_nodes)], dtype=torch.float)

### 3. Data Labelling


In [6]:
#data.y = torch.tensor([betweenness[i] for i in range(data.num_nodes)], dtype=torch.float)

**import from offline**

In [7]:
import pickle

file_path = '/kaggle/input/pubmed-graph-with-betweenness/pubmed.pickle'

# Carica il dataset dal file pickle
with open(file_path, 'rb') as f:
    data = pickle.load(f)

# Verifica che il dataset sia stato caricato correttamente
print(data)

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


### 4. Suddivisione Dataset in test e val


In [8]:
# nulla da fare, il modello cora è già suddiviso.

### 5. Sviluppo del modello


In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, BatchNorm

class GATRegressionOLD(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GATRegression, self).__init__()
        
        self.conv1 = GATConv(in_channels, hidden_channels, heads=8, dropout=0.6)
        self.bn1 = BatchNorm(hidden_channels * 8)
        
        self.conv2 = GATConv(hidden_channels * 8, hidden_channels, heads=8, dropout=0.6)
        self.bn2 = BatchNorm(hidden_channels * 8)
        
        self.conv3 = GATConv(hidden_channels * 8, out_channels, heads=1, concat=False, dropout=0.6)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.elu(x)
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.elu(x)
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv3(x, edge_index)
        
        return x


In [10]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GATRegression(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GATRegression, self).__init__()
        
        self.conv1 = GATConv(in_channels, hidden_channels, heads=8, dropout=0.6)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels * 8)
        
        self.conv2 = GATConv(hidden_channels * 8, hidden_channels, heads=8, dropout=0.6)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels * 8)
        
        self.conv3 = GATConv(hidden_channels * 8, out_channels, heads=1, concat=False, dropout=0.6)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.elu(x)
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.elu(x)
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv3(x, edge_index)
        
        # Assicurati che l'output sia positivo
        return F.softplus(x)


### 6. Training
#### Model setup

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

in_channels = dataset.num_node_features
hidden_channels = 64 
out_channels = 1  

model = GATRegression(in_channels, hidden_channels, out_channels).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


#### Training functions

In [12]:
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask].unsqueeze(1))
    loss.backward()
    optimizer.step()
    return loss.item()

def test(data):
    model.eval()
    with torch.no_grad():
        pred = model(data).squeeze()  # Previsioni
        loss = F.mse_loss(pred[data.test_mask], data.y[data.test_mask])
    return loss.item()


#### Training Phase

In [13]:
num_epochs = 4000

In [14]:
for epoch in range(1, num_epochs):
    loss = train(data)
    test_loss = test(data)
    print(f'Epoch: {epoch:03d}, Train Loss: {loss:.4f}, Test Loss: {test_loss:.4f}')

Epoch: 001, Train Loss: 0.6041, Test Loss: 0.4879
Epoch: 002, Train Loss: 0.3745, Test Loss: 0.4803
Epoch: 003, Train Loss: 2.2647, Test Loss: 0.4650
Epoch: 004, Train Loss: 0.5286, Test Loss: 0.4472
Epoch: 005, Train Loss: 0.4720, Test Loss: 0.4286
Epoch: 006, Train Loss: 0.3291, Test Loss: 0.4078
Epoch: 007, Train Loss: 0.5835, Test Loss: 0.3875
Epoch: 008, Train Loss: 0.2902, Test Loss: 0.3666
Epoch: 009, Train Loss: 0.3737, Test Loss: 0.3454
Epoch: 010, Train Loss: 0.8366, Test Loss: 0.3264
Epoch: 011, Train Loss: 0.3302, Test Loss: 0.3100
Epoch: 012, Train Loss: 0.2825, Test Loss: 0.2932
Epoch: 013, Train Loss: 0.3119, Test Loss: 0.2788
Epoch: 014, Train Loss: 0.2848, Test Loss: 0.2653
Epoch: 015, Train Loss: 1.8755, Test Loss: 0.2538
Epoch: 016, Train Loss: 0.2593, Test Loss: 0.2422
Epoch: 017, Train Loss: 0.7350, Test Loss: 0.2318
Epoch: 018, Train Loss: 0.2933, Test Loss: 0.2217
Epoch: 019, Train Loss: 0.6980, Test Loss: 0.2118
Epoch: 020, Train Loss: 0.3372, Test Loss: 0.2026


## Comparazione

In [18]:
from torch_geometric.datasets import KarateClub

# Carica il dataset Karate Club
dataset_k = KarateClub()
data_k = dataset_k[0]

import torch

def add_padding_to_features(data, target_num_features=512):
    #Numero di caratteristiche attuali
    num_features = data.x.shape[1]
    
    num_extra_features = target_num_features - num_features
    
    #Creo un tensore di zeri
    padding = torch.zeros((data.num_nodes, num_extra_features), dtype=torch.float)
    

    data.x = torch.cat([data.x, padding], dim=1)
    
    return data

data_k = add_padding_to_features(data_k, target_num_features=512)

print(f'Numero di caratteristiche per nodo dopo il padding: {data.x.shape[1]}')  # Deve restituire 512


data_k = data_k.to(device)

def test_model_on_karate(model, data):
    model.eval()  
    with torch.no_grad():  
        predictions = model(data).squeeze()
        return predictions.cpu().numpy()

predicted_betweenness = test_model_on_karate(model, data_k)


Numero di caratteristiche per nodo dopo il padding: 500


RuntimeError: mat1 and mat2 shapes cannot be multiplied (34x512 and 500x512)

In [ ]:
# Converti il grafo in formato NetworkX
G = nx.karate_club_graph()

# Calcola la betweenness centrality approssimata con l'algoritmo di Brandes
# Campioniamo 10 nodi per l'approssimazione
approx_betweenness = nx.betweenness_centrality(G, k=10, normalized=True)


In [ ]:
import pandas as pd

# Funzione per confrontare i risultati del modello con Brandes
def compare_with_brandes(predicted_betweenness, approx_betweenness):
    # Crea un DataFrame per confrontare le previsioni
    results = pd.DataFrame({
        'Node': range(len(predicted_betweenness)),
        'Predicted Betweenness': predicted_betweenness,
        'Approximate Betweenness (Brandes)': [approx_betweenness[i] for i in range(len(predicted_betweenness))]
    })
    
    # Mostra la tabella di confronto
    print(results)

# Confronta il modello con Brandes
compare_with_brandes(predicted_betweenness, approx_betweenness)


In [ ]:
import torch
import random

def compare_random_node(model, data):
    # Passa il modello in modalità di valutazione
    model.eval()
    
    # Seleziona un nodo casuale
    random_node = random.randint(0, data.num_nodes - 1)
    
    # Effettua la previsione per il nodo selezionato casualmente
    with torch.no_grad():
        prediction = model(data).squeeze()  # Previsioni di betweenness centrality per tutti i nodi
        predicted_value = prediction[random_node].item()
    
    # Recupera il valore esatto di betweenness centrality
    exact_value = data.y[random_node].item()
    
    # Stampa il risultato
    print(f"Confronto per il nodo {random_node}:")
    print(f"Valore predetto dal modello = {predicted_value:.4f}")
    print(f"Valore esatto di betweenness centrality = {exact_value:.4f}")

compare_random_node(model, data)


In [ ]:
# Funzione per confrontare i valori predetti con quelli esatti
def compare_predictions(model, data):
    # Passa il modello in modalità di valutazione
    model.eval()
    
    # Seleziona 10 nodi casuali
    num_nodes = data.num_nodes
    random_indices = torch.randperm(num_nodes)[:10]  # Seleziona 10 nodi casuali
    
    # Effettua le previsioni sui nodi selezionati
    with torch.no_grad():
        predictions = model(data).squeeze()  # Previsioni di betweenness centrality
        predicted_values = predictions[random_indices]
    
    # Recupera i valori esatti di betweenness centrality dai dati (data.y)
    exact_values = data.y[random_indices]
    
    # Mostra i risultati
    print("\nConfronto tra betweenness centrality predetta e reale:")
    for i, node in enumerate(random_indices):
        print(f" Nodo {node.item()}: Predetto = {predicted_values[i].item():.4f}, Reale = {exact_values[i].item():.4f}")
    
    # Calcola e ritorna l'errore medio quadratico
    mse = torch.mean((predicted_values - exact_values) ** 2).item()
    return mse

# Confronta le previsioni su 10 nodi casuali con i valori esatti
mse = compare_predictions(model, data)
print(f"\nErrore quadratico medio (MSE): {mse:.4f}")


In [ ]:
import pandas as pd

# Funzione per confrontare i 20 nodi con la betweenness centrality più alta
def compare_top_20_exact_vs_pred(model, data):
    # Passa il modello in modalità di valutazione
    model.eval()
    
    # Effettua le previsioni per tutti i nodi
    with torch.no_grad():
        predictions = model(data).squeeze()  # Previsioni di betweenness centrality
    
    # Seleziona i 20 nodi con le predizioni più alte
    _, topk_pred_indices = torch.topk(predictions, 20)
    
    # Seleziona i 20 nodi con la betweenness centrality più alta nei valori esatti
    _, topk_exact_indices = torch.topk(data.y, 20)
    
    # Crea un DataFrame per confrontare le posizioni
    results = pd.DataFrame({
        'Node (pred)': topk_pred_indices.cpu().numpy(),
        'Node (exact)': topk_exact_indices.cpu().numpy(),
    })
    
    # Aggiungi una colonna per verificare se la posizione corrisponde
    results['Correct'] = results['Node (pred)'] == results['Node (exact)']
    
    return results

# Confronta i 20 nodi con betweenness centrality più alta e mostra la tabella
results_df = compare_top_20_exact_vs_pred(model, data)

# Visualizza la tabella
print(results_df)
